In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [7]:
glob.glob(global_path + '/mod*_exp/**/result.csv', recursive=True)


['./outputs_permute/mod3_nc_nope_residual_exp/mod3_nc_sd240_T2405211834_nope_res=[3, 4, 5]_pmTrue_ncTrue/mod3_nc_sd240_T2405211834_nope_res=[3, 4, 5]_pmTrue_ncTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211220_res=[0, 3, 4, 5]_pmTrue/mods_sd240_T2405211220_res=[0, 3, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211230_res=[0, 1, 2, 4, 5]_pmTrue/mods_sd240_T2405211230_res=[0, 1, 2, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211228_res=[0, 1, 3, 4, 5]_pmTrue/mods_sd240_T2405211228_res=[0, 1, 3, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211214_res=[0, 4, 5]_pmTrue/mods_sd240_T2405211214_res=[0, 4, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211216_res=[0, 1, 5]_pmTrue/mods_sd240_T2405211216_res=[0, 1, 5]_pmTrue/result.csv',
 './outputs_permute/mods_residual_exp/mods_sd240_T2405211227_res=[0, 2, 3, 4, 5]_pmTrue/mods_sd240

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re

critical_keys1 = ['mod3', 'modp', 'mods']
critical_keys2 = ['nc']

global_path = './outputs_permute'

all_dfs = []

for c1 in critical_keys1:
    for c2 in critical_keys2:
        for c2_value in [True, False]:
            all_result_paths = glob.glob(global_path + '/mod*_exp/**/result.csv', recursive=True)
            all_result_paths = [p for p in all_result_paths if c1 in p]
            all_result_paths = [p for p in all_result_paths if (c2 in p)==c2_value]

            result_summary = {}
            n_layers = 6
            for result_path in all_result_paths:
                result = pd.read_csv(result_path)
                top_test_result = result['test_acc'].max()
                pe_type = 'NoPE' if 'nope' in result_path else 'Original'
                pattern = r"res=\[(.*?)\]"
                residual_list = re.findall(pattern, result_path)[0]
                residual_list = [int(num) for num in residual_list.split(", ")] if residual_list!='' else []
                # residual_list = [int(num) for num in residual_list.split(", ")]
                missing_rc = [i for i in range(n_layers) if i not in residual_list]
                missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


                if pe_type not in result_summary:
                    result_summary[pe_type] = {}
                if missing_rc_str not in result_summary[pe_type]:
                    result_summary[pe_type][missing_rc_str] = []
                result_summary[pe_type][missing_rc_str].append(top_test_result)
                if pe_type == 'Original' and missing_rc_str == '{5}':
                    print(result_path, pe_type, missing_rc_str, top_test_result)

            from ast import literal_eval
            mean_results = {}
            for pe_type in result_summary:
                cur_type_dict = result_summary[pe_type].copy()
                for missing_rc in cur_type_dict:
                    cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
                mean_results[pe_type] = cur_type_dict

            df = pd.DataFrame(mean_results)
            missing0 = ['{}']
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            missing6 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4), str(i+5)])+ '}'for i in range(n_layers-5)]
            from IPython.display import display
            # for missing in [missing0, missing1, missing2, missing3, missing4, missing5, missing6]:
            #     try:
            #         display(df.loc[missing,:].T)
            #     except:
            #         break
            print(c1, c2, c2_value, '='*100)
            display(df.T)
            
            if len(df) == 0:
                continue
            num_seeds = df.values.min()


            from collections import OrderedDict
            final_results = OrderedDict()
            for pe_type in {'Original', 'NoPE'}:
                cur_type_dict = result_summary[pe_type].copy()
                final_dict = {
                    # 'min': cur_type_dict.copy(),
                    'max': cur_type_dict.copy(),
                    'avg.': cur_type_dict.copy(),
                }
                for missing_rc in cur_type_dict:
                    final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
                    # final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
                    final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

                for k in final_dict:
                    final_results[pe_type+' ('+k+')'] = final_dict[k]

            final_results
            df = pd.DataFrame(final_results)
            # missing1 = [str({i}) for i in range(n_layers)]
            # missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            # missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            # missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            # missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            all_rs = []
            for missing in [missing0, missing1, missing2, missing3, missing4, missing5, missing6]:
                try:
                    all_rs.append(df.loc[missing,:].T)
                    display(df.loc[missing,:].T)
                except:
                    break
            # r1 = df.loc[missing1,:].T
            # r2 = df.loc[missing2,:].T
            # r3 = df.loc[missing3,:].T
            # r4 = df.loc[missing4,:].T
            # r5 = df.loc[missing5,:].T
            # r6 = df.loc[missing6,:].T
            # r0 = df.loc[['{}'],:].T
            # display(r0)
            # display(r1)
            # display(r2)
            # display(r3)
            # display(r4)
            # display(r5)
            # display(r6)

            # write all 3 to the same csv file
            all_r = pd.concat(all_rs, axis=1)
            c2 = '_' + c2 if c2_value else ''
            # all_r.to_csv(f'{c1}{c2}_{num_seeds}seeds.csv')
            all_dfs.append(all_r)
new_df = pd.concat(all_dfs, axis=0)
new_df.to_csv(f'./mod_all.csv')

./outputs_permute/mod3_nc_residual_exp/mod3_nc_sd240_T2405211822_res=[0, 1, 2, 3, 4]_pmTrue_ncTrue/mod3_nc_sd240_T2405211822_res=[0, 1, 2, 3, 4]_pmTrue_ncTrue/result.csv Original {5} 98.14
mod3 nc True ====================================================================================================


,"{0,1,2}",{0},"{0,1}","{3,4}",{1},"{3,4,5}","{1,2,3}","{2,3}","{4,5}",{3},{5},{4},"{2,3,4}","{1,2}",{},{2}
NoPE,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Original,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


,{}
NoPE (max),NaN
NoPE (avg.),NaN
Original (max),99.23
Original (avg.),99.23


,{0},{1},{2},{3},{4},{5}
NoPE (max),NaN,NaN,NaN,NaN,NaN,NaN
NoPE (avg.),NaN,NaN,NaN,NaN,NaN,NaN
Original (max),95.53,91.42,98.4,100.0,99.99,98.14
Original (avg.),95.53,91.42,98.4,100.0,99.99,98.14


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (max),NaN,NaN,NaN,NaN,NaN
NoPE (avg.),NaN,NaN,NaN,NaN,NaN
Original (max),97.18,92.87,99.81,94.3,100.0
Original (avg.),97.18,92.87,99.81,94.3,100.0


...

,"{1,2}",{3},{2},"{1,2,3}","{2,3,4}",{1},"{0,1}",{4},"{4,5}","{2,3}",{5},"{0,1,2}",{0},{},"{3,4}","{3,4,5}"
Original,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
NoPE,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


,{}
NoPE (max),99.4
NoPE (avg.),99.4
Original (max),100.0
Original (avg.),100.0


,{0},{1},{2},{3},{4},{5}
NoPE (max),100.0,100.0,100.0,100.0,100.0,99.8
NoPE (avg.),100.0,100.0,100.0,100.0,100.0,99.8
Original (max),100.0,100.0,100.0,99.8,100.0,100.0
Original (avg.),100.0,100.0,100.0,99.8,100.0,100.0


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (max),100.0,99.2,100.0,99.8,100.0
NoPE (avg.),100.0,99.2,100.0,99.8,100.0
Original (max),100.0,99.6,100.0,99.2,100.0
Original (avg.),100.0,99.6,100.0,99.2,100.0


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (max),100.0,86.6,100.0,100.0
NoPE (avg.),100.0,86.6,100.0,100.0
Original (max),90.2,33.6,99.2,100.0
Original (avg.),90.2,33.6,99.2,100.0


In [20]:
print('\\begin{tabular}{lllrrrrrrrrrrrrrrrrrrrrrr}\n\\toprule\nUnnamed: 0 & Unnamed: 1 &      Unnamed: 2 &    \\{\\} &   \\{0\\} &   \\{1\\} &   \\{2\\} &   \\{3\\} &   \\{4\\} &   \\{5\\} &  \\{0,1\\} &  \\{1,2\\} &  \\{2,3\\} &  \\{3,4\\} &  \\{4,5\\} &  \\{0,1,2\\} &  \\{1,2,3\\} &  \\{2,3,4\\} &  \\{3,4,5\\} &  \\{0,1,2,3\\} &  \\{1,2,3,4\\} &  \\{2,3,4,5\\} &  \\{0,1,2,3,4\\} &  \\{1,2,3,4,5\\} &  \\{0,1,2,3,4,5\\} \\\\\n\\midrule\n   mod3(5) & non-causal &      NoPE (max) &  96.9 &  98.8 & 100.0 & 100.0 & 100.0 & 100.0 &  99.2 &   98.4 &  100.0 &  100.0 &  100.0 &  100.0 &     99.5 &     97.7 &    100.0 &    100.0 &       94.0 &       96.6 &       99.3 &         92.4 &         99.4 &           91.9 \\\\\n       NaN &        NaN &     NoPE (avg.) &  93.9 &  94.2 &  97.3 &  98.5 &  98.9 &  98.2 &  95.4 &   96.1 &   94.8 &   98.4 &   97.9 &   97.4 &     95.7 &     89.7 &     99.6 &     98.0 &       86.4 &       89.4 &       97.6 &         88.1 &         82.2 &           83.2 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &   99.9 &  100.0 &  100.0 &  100.0 &  100.0 &     99.2 &     91.0 &    100.0 &    100.0 &       88.9 &       90.4 &       97.7 &         94.6 &         89.1 &           90.9 \\\\\n       NaN &        NaN & Original (avg.) &  99.2 &  94.3 &  96.1 &  99.6 &  99.4 &  99.0 &  95.0 &   97.5 &   93.9 &   99.6 &   99.8 &   97.7 &     94.9 &     85.8 &     99.5 &    100.0 &       85.9 &       77.6 &       87.8 &         87.2 &         82.3 &           84.3 \\\\\n       NaN &     causal &      NoPE (max) & 100.0 & 100.0 &  99.5 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &   99.9 &  100.0 &  100.0 &  100.0 &    100.0 &     97.9 &     99.2 &     34.0 &       94.9 &       34.6 &       34.0 &         34.6 &         34.2 &           34.3 \\\\\n       NaN &        NaN &     NoPE (avg.) &  99.8 &  99.6 &  87.9 &  99.6 &  99.8 &  99.7 &  99.9 &   99.1 &   97.8 &   99.4 &   99.9 &   99.9 &     99.8 &     90.9 &     92.4 &     33.7 &       86.4 &       33.8 &       33.7 &         33.8 &         33.7 &           33.8 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &   99.8 &  100.0 &  100.0 &  100.0 &    100.0 &     99.4 &     98.7 &     34.2 &       96.2 &       34.4 &       34.2 &         34.0 &         34.2 &           34.2 \\\\\n       NaN &        NaN & Original (avg.) &  99.8 &  99.4 &  97.0 &  99.7 &  99.7 &  99.8 &  99.9 &   99.8 &   98.3 &   99.7 &   99.9 &   99.9 &     99.2 &     93.0 &     92.6 &     33.8 &       74.8 &       33.8 &       33.9 &         33.7 &         33.8 &           33.8 \\\\\n   mod3(3) & non-causal &      NoPE (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &      100.0 &        100.0 &        100.0 &          100.0 \\\\\n       NaN &        NaN &     NoPE (avg.) & 100.0 & 100.0 & 100.0 &  99.9 & 100.0 &  99.9 &  99.9 &  100.0 &  100.0 &   99.9 &   99.9 &  100.0 &    100.0 &     99.9 &     99.7 &     99.9 &      100.0 &       99.5 &      100.0 &         99.6 &         99.9 &           99.9 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &      100.0 &        100.0 &        100.0 &          100.0 \\\\\n       NaN &        NaN & Original (avg.) &  99.9 &  99.9 & 100.0 &  99.9 & 100.0 & 100.0 &  99.9 &   99.9 &  100.0 &   99.9 &  100.0 &  100.0 &     99.8 &     99.8 &    100.0 &     99.6 &       99.9 &       99.4 &       99.0 &         99.8 &         98.8 &           99.7 \\\\\n       NaN &     causal &      NoPE (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &       36.6 &       36.2 &         36.0 &         36.2 &           36.2 \\\\\n       NaN &        NaN &     NoPE (avg.) &  99.9 &  99.9 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &   99.9 &   99.6 &   99.1 &   99.2 &   99.4 &     99.8 &     95.0 &     95.5 &     76.2 &       83.6 &       35.1 &       34.1 &         34.4 &         34.3 &           34.3 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &     97.0 &       99.8 &       36.6 &       36.6 &         39.6 &         36.6 &           36.6 \\\\\n       NaN &        NaN & Original (avg.) & 100.0 & 100.0 & 100.0 & 100.0 &  99.9 & 100.0 &  99.9 &   99.8 &  100.0 &   99.6 &   99.7 &   99.9 &     99.9 &     99.6 &     97.9 &     74.2 &       89.5 &       34.4 &       34.4 &         34.8 &         34.6 &           34.5 \\\\\n  mod3*(5) & non-causal &      NoPE (max) &  47.7 &  47.6 &  48.2 &  48.2 &  48.3 &  48.0 &  48.0 &   48.0 &   48.2 &   48.3 &   48.0 &   48.3 &     48.2 &     48.3 &     48.1 &     48.4 &       48.1 &       48.0 &       48.4 &         48.3 &         48.1 &           48.3 \\\\\n       NaN &        NaN &     NoPE (avg.) &  47.2 &  47.2 &  47.7 &  48.0 &  47.9 &  47.5 &  47.3 &   47.3 &   48.0 &   47.8 &   47.8 &   48.0 &     47.6 &     47.8 &     47.8 &     48.1 &       47.4 &       47.8 &       47.9 &         47.8 &         47.9 &           47.9 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &      100.0 &        100.0 &        100.0 &          100.0 \\\\\n       NaN &        NaN & Original (avg.) &  99.9 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &   99.9 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &       99.9 &        100.0 &        100.0 &          100.0 \\\\\n       NaN &     causal &      NoPE (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &      100.0 &         99.9 &         89.6 &           90.1 \\\\\n       NaN &        NaN &     NoPE (avg.) & 100.0 & 100.0 & 100.0 &  99.9 & 100.0 & 100.0 & 100.0 &   99.8 &  100.0 &  100.0 &  100.0 &   99.9 &     99.8 &     99.9 &     99.9 &    100.0 &       99.8 &       94.0 &       95.5 &         85.2 &         61.3 &           63.8 \\\\\n       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &    100.0 &    100.0 &      100.0 &      100.0 &      100.0 &         98.6 &         97.4 &           34.4 \\\\\n       NaN &        NaN & Original (avg.) & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 &  100.0 &  100.0 &  100.0 &  100.0 &  100.0 &    100.0 &    100.0 &     99.9 &    100.0 &       98.7 &       93.3 &       96.8 &         62.6 &         51.3 &           33.8 \\\\\n\\bottomrule\n\\end{tabular}\n')

\begin{tabular}{lllrrrrrrrrrrrrrrrrrrrrrr}
\toprule
Unnamed: 0 & Unnamed: 1 &      Unnamed: 2 &    \{\} &   \{0\} &   \{1\} &   \{2\} &   \{3\} &   \{4\} &   \{5\} &  \{0,1\} &  \{1,2\} &  \{2,3\} &  \{3,4\} &  \{4,5\} &  \{0,1,2\} &  \{1,2,3\} &  \{2,3,4\} &  \{3,4,5\} &  \{0,1,2,3\} &  \{1,2,3,4\} &  \{2,3,4,5\} &  \{0,1,2,3,4\} &  \{1,2,3,4,5\} &  \{0,1,2,3,4,5\} \\
\midrule
   mod3(5) & non-causal &      NoPE (max) &  96.9 &  98.8 & 100.0 & 100.0 & 100.0 & 100.0 &  99.2 &   98.4 &  100.0 &  100.0 &  100.0 &  100.0 &     99.5 &     97.7 &    100.0 &    100.0 &       94.0 &       96.6 &       99.3 &         92.4 &         99.4 &           91.9 \\
       NaN &        NaN &     NoPE (avg.) &  93.9 &  94.2 &  97.3 &  98.5 &  98.9 &  98.2 &  95.4 &   96.1 &   94.8 &   98.4 &   97.9 &   97.4 &     95.7 &     89.7 &     99.6 &     98.0 &       86.4 &       89.4 &       97.6 &         88.1 &         82.2 &           83.2 \\
       NaN &        NaN &  Original (max) & 100.0 & 100.0 & 100.0 &

In [ ]:
# This is non-causal trained on mod3, 2 seeds